In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn import preprocessing
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
y_train = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Roberta/embeddings/train_labels.csv")
y_test = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Roberta/embeddings/test_labels.csv")


In [3]:
with open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Roberta/embeddings/train_roberta_embeddings_target_.pkl', 'rb') as f:
    x_train = pickle.load(f, encoding='latin1')

with open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Roberta/embeddings/test_roberta_embeddings_target_.pkl', 'rb') as f:
    x_test = pickle.load(f, encoding='latin1')
    

In [4]:
x_train_vals = []
for sample in x_train["embeddings"]:
    x_train_vals.append(torch.stack(sample))
    
x_test_vals = []
for sample in x_test["embeddings"]:
    x_test_vals.append(torch.stack(sample))

x_train_df = pd.DataFrame({'embeddings':x_train_vals})
x_test_df = pd.DataFrame({'embeddings':x_test_vals})

x_train_df["sarcasm"] = y_train["sarcasm"]
x_test_df["sarcasm"] = y_test["sarcasm"]
x_train_df["sarcasm"] = x_train_df["sarcasm"].astype('int').to_numpy()
x_test_df["sarcasm"] = x_test_df["sarcasm"].astype('int').to_numpy()

In [5]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'embeddings']
            features = np.array(features)
            a=np.empty((512,1))
            a.fill(self.data.loc[index, 'speaker'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'embeddings']
            features = np.array(features)
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label

    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
class GRUNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 50
output_size = 2
input_size_sid = 768
input_size_sd = 769
n_layers = 1

### Speaker Independent and Context Independent

In [6]:
gru_train_tensor = GRUTensorDataset(x_train_df[['embeddings', 'sarcasm']], False)
gru_test_tensor = GRUTensorDataset(x_test_df[['embeddings', 'sarcasm']], False)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(test_indices)
)

In [7]:
gru = GRUNetSID(input_size_sid, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSID(
  (gru): GRU(768, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [8]:
n_epochs = 41
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)

Epoch: 0
              precision    recall  f1-score     support
0              0.644068  0.314050  0.422222  121.000000
1              0.543956  0.825000  0.655629  120.000000
accuracy       0.568465  0.568465  0.568465    0.568465
macro avg      0.594012  0.569525  0.538926  241.000000
weighted avg   0.594220  0.568465  0.538441  241.000000
Epoch: 20
              precision    recall  f1-score     support
0              0.588235  0.578512  0.583333  121.000000
1              0.581967  0.591667  0.586777  120.000000
accuracy       0.585062  0.585062  0.585062    0.585062
macro avg      0.585101  0.585090  0.585055  241.000000
weighted avg   0.585114  0.585062  0.585048  241.000000
Epoch: 40
              precision    recall  f1-score    support
0              0.657895  0.413223  0.507614  121.00000
1              0.569697  0.783333  0.659649  120.00000
accuracy       0.597510  0.597510  0.597510    0.59751
macro avg      0.613796  0.598278  0.583632  241.00000
weighted avg   0.613979 

### Speaker Dependent and Context Independent

In [14]:
x_train_speakers = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Roberta/embeddings/train_data.csv")
x_test_speakers = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Roberta/embeddings/test_data.csv")
x_train_speakers

,target_,target_context,speaker
0,<s> i have been told it is a good way to move...,<s> i have been told it is a good way to move...,25
1,"<s> yeah, sure. you slept with your husband. ...","<s> yeah, sure. you slept with your husband. ...",1
2,<s> when are you coming home? </s>,<s> when are you coming home? okay. alright....,16
3,<s> riveting. </s>,<s> riveting. bingo. then i lifted the cushi...,0
4,"<s> no, this is just part of a daredevil game...","<s> no, this is just part of a daredevil game...",2
...,...,...,...
956,"<s> oh, that is sweet, but today is all about...","<s> oh, that is sweet, but today is all about...",7
957,<s> if you want to put a label on it. </s>,<s> if you want to put a label on it. you me...,24
958,<s> that you are an alcoholic? </s>,<s> that you are an alcoholic? i realized so...,3
959,<s> all i see is a yellow smudge. </s>,<s> all i see is a yellow smudge. now go bac...,15


In [15]:
x_train_df["speaker"] = x_train_speakers["speaker"]
x_test_df["speaker"] = x_test_speakers["speaker"]
x_train_df

,embeddings,sarcasm,speaker
0,"[[tensor(-0.0328), tensor(0.0243), tensor(-0.0...",0,25
1,"[[tensor(-0.0586), tensor(0.0241), tensor(-0.0...",0,1
2,"[[tensor(-0.0426), tensor(0.0327), tensor(-0.0...",0,16
3,"[[tensor(-0.0465), tensor(0.0277), tensor(-0.0...",1,0
4,"[[tensor(-0.0368), tensor(0.0129), tensor(-0.0...",1,2
...,...,...,...
956,"[[tensor(-0.0408), tensor(0.0372), tensor(-0.0...",0,7
957,"[[tensor(-0.0562), tensor(0.0277), tensor(-0.0...",1,24
958,"[[tensor(-0.0423), tensor(0.0244), tensor(-0.0...",1,3
959,"[[tensor(-0.0483), tensor(0.0243), tensor(-0.0...",0,15


In [16]:
gru_train_tensor = GRUTensorDataset(x_train_df[['embeddings', 'sarcasm', 'speaker']], True)
gru_test_tensor = GRUTensorDataset(x_test_df[['embeddings', 'sarcasm', 'speaker']], True)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    drop_last = True,
    sampler=SubsetRandomSampler(test_indices)
)

In [17]:
gru = GRUNetSID(input_size_sd, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSID(
  (gru): GRU(769, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [18]:
n_epochs = 31
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target.long())
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%10 == 0):
        print("Epoch: " + str(epoch))
        test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)

Epoch: 0
              precision    recall  f1-score     support
0              0.634783  0.603306  0.618644  121.000000
1              0.619048  0.650000  0.634146  120.000000
accuracy       0.626556  0.626556  0.626556    0.626556
macro avg      0.626915  0.626653  0.626395  241.000000
weighted avg   0.626948  0.626556  0.626363  241.000000
Epoch: 10
              precision    recall  f1-score     support
0              0.627451  0.528926  0.573991  121.000000
1              0.589928  0.683333  0.633205  120.000000
accuracy       0.605809  0.605809  0.605809    0.605809
macro avg      0.608690  0.606129  0.603598  241.000000
weighted avg   0.608767  0.605809  0.603475  241.000000
Epoch: 20
              precision    recall  f1-score     support
0              0.654545  0.595041  0.623377  121.000000
1              0.625954  0.683333  0.653386  120.000000
accuracy       0.639004  0.639004  0.639004    0.639004
macro avg      0.640250  0.639187  0.638382  241.000000
weighted avg   0.64